# Debug Drill: The Too-Good Model

**Scenario:**
A colleague built a churn prediction pipeline. They're excited because the model achieves 97% AUC on the test set.

"This is the best model we've ever built!" they say. "We should ship it immediately!"

**Your Task:**
1. Run the pipeline end-to-end
2. Find the bug (hint: it's NOT in the model training step)
3. Fix it
4. Write a 3-bullet postmortem

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report

np.random.seed(42)

In [ ]:
# ===== STAGE 1: DATA LOADING =====
DATA_URL = 'https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/'

df = pd.read_csv(DATA_URL + 'streamcart_customers.csv')
print(f"Loaded {len(df):,} customers")
print(f"Churn rate: {df['churned'].mean():.1%}")

In [ ]:
# ===== STAGE 2: FEATURE ENGINEERING =====
# Colleague's feature engineering code

def engineer_features(data):
    """Create features for churn prediction."""
    df_feat = data.copy()
    
    # Behavioral features
    df_feat['orders_per_month'] = df_feat['orders_total'] / (df_feat['tenure_months'] + 1)
    df_feat['spend_per_order'] = df_feat['total_spend'] / (df_feat['orders_total'] + 1)
    df_feat['login_intensity'] = df_feat['logins_last_30d'] / 30
    
    # Engagement ratio
    df_feat['engagement_ratio'] = df_feat['logins_per_month_avg'] / (df_feat['orders_per_month'] + 0.1)
    
    # Recency features
    df_feat['days_since_last_order'] = df_feat['days_since_last_purchase']
    
    # Support interaction
    df_feat['tickets_per_tenure'] = df_feat['support_tickets_total'] / (df_feat['tenure_months'] + 1)
    
    return df_feat

df_engineered = engineer_features(df)
print(f"Engineered {len(df_engineered.columns)} columns")

In [ ]:
# ===== STAGE 3: PREPROCESSING =====
# Colleague's preprocessing code (CONTAINS BUG)

feature_cols = [
    'tenure_months', 'orders_per_month', 'spend_per_order', 
    'login_intensity', 'engagement_ratio', 'days_since_last_order',
    'tickets_per_tenure', 'avg_order_value'
]

X = df_engineered[feature_cols].fillna(0)
y = df_engineered['churned']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # <-- Colleague scales ALL data here

print(f"Features shape: {X_scaled.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")

In [ ]:
# ===== STAGE 4: TRAIN/TEST SPLIT =====
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train size: {len(X_train):,}")
print(f"Test size: {len(X_test):,}")

In [ ]:
# ===== STAGE 5: MODEL TRAINING =====
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

print("Model trained!")

In [ ]:
# ===== STAGE 6: EVALUATION =====
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)

print(f"Test AUC: {auc:.3f}")
print(f"\n🎉 Colleague: 'This is amazing! 97% AUC! Ship it!'")

---

## Your Investigation

The AUC is suspiciously high. Something is wrong.

### Step 1: Which pipeline stage contains the bug?

Remember the ML pipeline stages:
1. Problem Framing
2. Data Collection  
3. Feature Engineering
4. Preprocessing
5. Train/Test Split
6. Model Training
7. Evaluation

In [ ]:
# TODO: Investigate each stage
# Look at the code above. What happens BEFORE the train/test split?

# Hint 1: Look at the order of operations
print("Order of operations:")
print("1. Load data")
print("2. Engineer features")
print("3. Scale features with StandardScaler")
print("4. Split into train/test")
print("5. Train model")
print("6. Evaluate")

print("\n🔍 What's wrong with this order?")

In [ ]:
# Hint 2: What does StandardScaler.fit_transform() learn from the data?
print("StandardScaler learns:")
print(f"  Mean of each feature: {scaler.mean_[:3]}...")
print(f"  Std of each feature: {scaler.scale_[:3]}...")

print("\n🤔 Question: When scaler.fit_transform(X) was called, what data was it using?")
print("   Answer: ALL the data — including what would become the test set!")

In [ ]:
# Write your diagnosis:

diagnosis = """
YOUR DIAGNOSIS HERE:

The bug is in stage: _______________

The problem is: _______________

This is called: _______________

Why it causes inflated AUC: _______________

"""
print(diagnosis)

### Step 2: Fix the pipeline

In [ ]:
# TODO: Implement the correct pipeline order

# The key insight: Split FIRST, then fit scaler only on training data

# Uncomment and complete:

# # Correct order:
# # 1. Split raw features FIRST
# X_raw = df_engineered[feature_cols].fillna(0)
# y = df_engineered['churned']
#
# X_train_raw, X_test_raw, y_train_fixed, y_test_fixed = train_test_split(
#     X_raw, y, test_size=0.2, random_state=42, stratify=y
# )
#
# # 2. Fit scaler ONLY on training data
# scaler_fixed = StandardScaler()
# X_train_fixed = scaler_fixed.fit_transform(X_train_raw)  # fit + transform on train
# X_test_fixed = scaler_fixed.transform(X_test_raw)        # transform only on test
#
# # 3. Train model
# model_fixed = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
# model_fixed.fit(X_train_fixed, y_train_fixed)
#
# # 4. Evaluate
# y_pred_proba_fixed = model_fixed.predict_proba(X_test_fixed)[:, 1]
# auc_fixed = roc_auc_score(y_test_fixed, y_pred_proba_fixed)
#
# print(f"Fixed Test AUC: {auc_fixed:.3f}")
# print(f"\nDifference: {auc - auc_fixed:.3f} points lower")

In [ ]:
# ============================================
# SELF-CHECK: Did you fix it?
# ============================================

# Uncomment after fixing:
#
# # The fixed AUC should be lower (more realistic)
# assert auc_fixed < auc, "Fixed AUC should be lower than the leaky AUC!"
# 
# # The difference should be noticeable
# assert auc - auc_fixed > 0.01, "Should see at least 1% drop in AUC"
#
# print("✓ Pipeline fixed!")
# print(f"✓ Buggy AUC: {auc:.3f} (inflated by preprocessing leakage)")
# print(f"✓ Fixed AUC: {auc_fixed:.3f} (realistic estimate)")

### Step 3: Understand why this matters

Preprocessing leakage is subtle because:
1. The code runs without errors
2. The model trains successfully  
3. The metrics look great
4. **But production performance will be worse**

In production:
- You can only use statistics from past data
- The scaler must be fit on training data and saved
- New data gets transformed using the saved scaler

When you fit the scaler on ALL data (including test), you're:
- Using future information (test set statistics)
- Making the test set "easier" to predict
- Getting an optimistic estimate of production performance

### Step 4: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Too-Good Model

### What happened:
- (Your answer: What was the symptom that indicated a problem?)

### Root cause:
- (Your answer: Which pipeline stage had the bug? What exactly was wrong?)

### How to prevent:
- (Your answer: What checks would catch this in future pipelines?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Pipeline order matters.** Split before preprocessing, not after.

2. **Preprocessing leakage is silent.** No errors, no warnings — just inflated metrics.

3. **The bug was in Stage 3-4 boundary.** The model training (Stage 5) was fine. Most ML bugs are at stage boundaries, not in the model itself.

4. **"Too good to be true" is a red flag.** Domain-appropriate AUC for churn is typically 0.70-0.85. Getting 0.97 should trigger investigation.

---

## Bonus: Other preprocessing leakage patterns

In [ ]:
# Pattern 1: Imputation leakage
print("WRONG:")
print("  df['feature'] = df['feature'].fillna(df['feature'].mean())  # Uses all data")
print("  X_train, X_test = train_test_split(...)")

print("\nRIGHT:")
print("  X_train, X_test = train_test_split(...)  # Split first")
print("  train_mean = X_train['feature'].mean()  # Calculate on train only")
print("  X_train['feature'] = X_train['feature'].fillna(train_mean)")
print("  X_test['feature'] = X_test['feature'].fillna(train_mean)  # Use train mean")

In [ ]:
# Pattern 2: Encoding leakage
print("WRONG:")
print("  encoder.fit(df['category'])  # Learns all categories")
print("  X_train, X_test = train_test_split(...)")

print("\nRIGHT:")
print("  X_train, X_test = train_test_split(...)  # Split first")
print("  encoder.fit(X_train['category'])  # Learn from train only")
print("  # Handle unknown categories in test with 'handle_unknown'")

In [ ]:
# Pattern 3: Feature selection leakage
print("WRONG:")
print("  # Select features using correlation with target on ALL data")
print("  correlations = df[features].corrwith(df['target'])")
print("  best_features = correlations.nlargest(10).index")
print("  X_train, X_test = train_test_split(df[best_features], ...)")

print("\nRIGHT:")
print("  # Split first, select features using train only")
print("  X_train, X_test = train_test_split(...)")
print("  correlations = X_train[features].corrwith(y_train)")
print("  best_features = correlations.nlargest(10).index")

---

## The Universal Rule

**Anything that "learns" from data must learn only from training data.**

This includes:
- Scalers (mean, std)
- Imputers (fill values)
- Encoders (category mappings)
- Feature selectors (importance scores)
- Dimensionality reducers (PCA components)

**The fix is always the same:** Split first, fit on train, transform both.